## Pipeline

[tensorflow-object-detection-api-tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/)

### Imports

In [1]:
import os
import pandas as pd
import json
import shutil

[TensorFlow SLIM](https://github.com/google-research/tf-slim)

In [2]:
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 4.7 MB/s 


### Modify label map

In [3]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

/content
/content/models
/content/models/research


### Get model

[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [4]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz \
-O /content/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz

--2022-10-18 23:57:15--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.219.128, 2607:f8b0:4001:c13::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.219.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470656289 (449M) [application/x-tar]
Saving to: ‘/content/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz’

/content/faster_rcn 100%[===================>] 448.85M   158MB/s    in 2.8s    

2022-10-18 23:57:18 (158 MB/s) - ‘/content/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz’ saved [470656289/470656289]



### Unzid model

In [5]:
!tar -zxvf /content/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
output_path = 'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almacenó en {}".format(output_path))

faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/pipeline.config
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/saved_model/
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/saved_model/saved_model.pb
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/saved_model/variables/
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/saved_model/variables/variables.index
La carpeta se almacenó en /content/models/research/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8


In [6]:
path_training = '/content/faster_rcnn'
os.mkdir(path_training)

### Copy config file

In [7]:
source_config = "{}/pipeline.config".format(output_path)
target_config = "{}/pipeline.config".format(path_training)
shutil.copyfile(source_config, target_config)#mueve el archivo de la carpeta A a la carpeta B

'/content/faster_rcnn/pipeline.config'

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
config = config_util.get_configs_from_pipeline_file(target_config)

In [10]:
config

{'model': faster_rcnn {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 640
       max_dimension: 640
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "faster_rcnn_resnet152_keras"
     batch_norm_trainable: true
   }
   first_stage_anchor_generator {
     grid_anchor_generator {
       height_stride: 16
       width_stride: 16
       scales: 0.25
       scales: 0.5
       scales: 1.0
       scales: 2.0
       aspect_ratios: 0.5
       aspect_ratios: 1.0
       aspect_ratios: 2.0
     }
   }
   first_stage_box_predictor_conv_hyperparams {
     op: CONV
     regularizer {
       l2_regularizer {
         weight: 0.0
       }
     }
     initializer {
       truncated_normal_initializer {
         stddev: 0.009999999776482582
       }
     }
   }
   first_stage_nms_score_threshold: 0.0
   first_stage_nms_iou_threshold: 0.699999988079071
   first_stage_max_proposals: 300
   first_stage_localization_loss_weight: 2.

In [11]:
#Se hace para editar este archivo
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [12]:
pipeline_config

model {
  faster_rcnn {
    num_classes: 90
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "faster_rcnn_resnet152_keras"
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 16
        width_stride: 16
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.009999999776482582
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.699999988079071
    first_stage_max_proposals: 300
  

### TFRecord and label_map Path

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
label_map_pbtxt_fname = "/content/drive/MyDrive/Parcial2/TFRecord/animals_label_map.pbtxt"
train_record_fname = "/content/drive/MyDrive/Parcial2/TFRecord/train/animals.tfrecord"
test_record_fname = "/content/drive/MyDrive/Parcial2/TFRecord/valid/animals.tfrecord"

### Modify config file

In [15]:
pipeline_config.model.ssd.num_classes = 6
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = "{}/checkpoint/ckpt-0".format(output_path)
# pipeline_config.train_config.fine_tune_checkpoint = "/content/drive/MyDrive/Olympia/Models_saved/checkpoints/ckpt-63"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [16]:
pipeline_config

model {
  ssd {
    num_classes: 6
  }
}
train_config {
  batch_size: 1
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.03999999910593033
          total_steps: 25000
          warmup_learning_rate: 0.013333000242710114
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/models/research/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  num_steps: 25000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  use_bfloat16: true
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "/content/drive/MyDrive/Parcial2/TFRecord/animals_label_map.pbtxt"
  tf_record_in

In [17]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
  f.write(config_text) 

- LVIS API enables reading and interacting with annotation files, visualizing annotations, and evaluating results.
- The TensorFlow official models are a collection of models that use TensorFlow’s high-level APIs. They are intended to be well-maintained, tested, and kept up to date with the latest TensorFlow API. They should also be reasonably optimized for fast performance while still being easy to read.

In [18]:
!pip install lvis
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 99 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.1 MB/s 
     |████████████████████████████████| 38.2 MB 1.2 MB/s 
     |████████████████████████████████| 5.9 MB 62.3 MB/s 
     |████████████████████████████████| 116 kB 62.8 MB/s 
     |████████████████████████████████| 636 kB 65.1 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 238 kB 52.4 MB/s 
     |████████████████████████████████| 1.7 MB 45.4 MB/s 
     |████████████████████████████████| 5.9 MB 45.6 MB/s 
     |████████████████████████████████| 438 kB 62.3 M

In [19]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.0 MB 59.6 MB/s 


In [ ]:
# !pip install lvis
# !pip install tf-models-official2.7.0

In [20]:
!dir

adversarial_text			       lfads
attention_ocr				       lstm_object_detection
audioset				       marco
autoaugment				       nst_blogpost
cognitive_planning			       object_detection
cvt_text				       pcl_rl
deeplab					       README.md
deep_speech				       rebar
delf					       seq_flow_lite
efficient-hrl				       slim
faster_rcnn_resnet152_v1_640x640_coco17_tpu-8  vid2depth


### Train model

In [21]:
num_steps = 1000
model_dir = "/content/drive/MyDrive/Parcial2/Models_saved"

!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_train_steps={num_steps}

2022-10-19 00:01:46.945099: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-19 00:01:47.931452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-19 00:01:47.931630: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-10-19 00:01:47.931652: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p

**DICE QUE LA VERSION DE TENSORFLOW NO ES COMPATIBLE CON EL EXTRACTOR DE CARACTERISTICAS**

## Results

### Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Parcial2/Models_saved"# "/content/ssd_mobilenet"

### Save model

In [ ]:
# model_dir = "/content/drive/MyDrive/Olympia/Models_saved/checkpoints"
# output_directory = "/content/drive/MyDrive/Olympia/Models_saved/Model_2"
# !python /content/models/research/object_detection/exporter_main_v2.py \
# --input_type image_tensor \
# --pipeline_config_path {target_config} \
# --trained_checkpoint_dir {model_dir} \
# --output_directory {output_directory}

2022-09-16 15:30:00.389152: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0916 15:30:00.894883 140503726458752 deprecation.py:628] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0916 15:30:21.529000 140503726458752 save_impl.py:72] Skipping full serialization of Ker

In [ ]:
!zip -r /content/drive/MyDrive/Olympia/Models_saved/Model_2/fine_tuned_model2.zip /content/drive/MyDrive/Olympia/Models_saved/Model_2

updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ckpt-0.data-00000-of-00001 (deflated 8%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/ckpt-0.index (deflated 80%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/checkpoint/checkpoint (deflated 40%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/ (stored 0%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/variables.data-00000-of-00001 (deflated 9%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/variables/variables.index (deflated 78%)
updating: content/drive/MyDrive/Olympia/Models_saved/Model_2/saved_model/assets/ (stored 0%)
updating: cont